# IMPORT

In [1]:
from pybit.unified_trading import HTTP
from datetime import datetime
import math
import talib
import numpy as np

# API KEY

In [1]:
# 파일을 읽기 모드로 열기
with open("C:\\Users\\lee\\Desktop\\bybit\\api.txt", "r") as file:
    api_list = []
    # 파일의 각 줄에 대해 반복
    for line in file:
        # 각 줄 출력
        api_list.append(line.strip())  # strip()은 줄 바꿈 문자(\n) 등을 제거하여 출력합니다.

key = api_list[0]
secret = api_list[1]

# 트레이딩 정보 가져오기 / 교차, 격리, 레버리지 배율 수정

In [2]:
# api 적용 세션
session = HTTP(
    # testnet=True,
    api_key=key,
    api_secret=secret,
)

# 트레이딩 정보 가져오기
trading_info = session.get_positions(
    category="linear",
    symbol="BTCUSDT",
)

cross_isolated = trading_info['result']['list'][0]['tradeMode']

if cross_isolated == 1:
    print('모드 : 격리')
else:
    # 교차, 격리 / 레버리지 배율 수정
    session.switch_margin_mode(
    category="linear",
    symbol="MNTUSDT",
    tradeMode=1, # 0: cross margin. 1: isolated margin
    buyLeverage="1",
    sellLeverage="1",
    )
    print('모드 : 격리로 변경')



모드 : 격리


# 캔들 마크 가격

In [7]:
now = datetime.now()
now_timestamp = math.floor(now.timestamp()*1000)

# get_kline / get_mark_price_kline
session = HTTP(testnet=True)
price = session.get_mark_price_kline(
    category="linear",
    symbol="BTCUSDT",
    interval=1,
    start=now_timestamp-60000000,
    end=now_timestamp,
    limit=1000,
)

price['result']['list'][0:3]

[['1700733720000', '37605.69', '37608.1', '37595.98', '37599.66'],
 ['1700733660000', '37638.9', '37638.9', '37605.69', '37605.69'],
 ['1700733600000', '37642.6', '37644.41', '37638.2', '37638.9']]

In [ ]:
a_hundred_prices = price['result']['list']
highPrice = []
lowPrice = []
closePrice = []
for i in range(len(a_hundred_prices)):
    # a = float(a_hundred_prices[i][2])
    # b = float(a_hundred_prices[i][3])
    # c = float(a_hundred_prices[i][4])
    # highPrice.append(a)
    # lowPrice.append(b)
    # closePrice.append(c)

    highPrice.append(float(a_hundred_prices[i][2]))
    lowPrice.append(float(a_hundred_prices[i][3]))
    closePrice.append(float(a_hundred_prices[i][4]))

# 가장 최신 가격을 앞에서 뒤로 변경하기 위한 코드
highPrice_reverse = highPrice[::-1]
lowPrice_reverse = lowPrice[::-1]
closePrice_reverse = closePrice[::-1]

# 기술적 지표

In [ ]:
def ema_indicator(closePrice_reverse, period):
    # EMA
    price_data = np.array(closePrice_reverse) # 파라미터

    ema_period = period # 파라미터
    ema_values = talib.EMA(price_data, timeperiod=ema_period)

    return ema_values

In [ ]:
def rsi_indicator(closePrice_reverse, period):
    # RSI
    price_data = np.array(closePrice_reverse) # 파라미터

    rsi_period = period # 파라미터
    rsi_values = talib.RSI(price_data, timeperiod=rsi_period)

    return rsi_values

In [ ]:
def adx_indicator(highPrice_reverse, lowPrice_reverse, closePrice_reverse, period):
    # ADX
    high_data = np.array(highPrice_reverse) # 파라미터
    low_data = np.array(lowPrice_reverse) # 파라미터
    close_data = np.array(closePrice_reverse) # 파라미터

    adx_period = period # 파라미터
    adx_values = talib.ADX(high_data, low_data, close_data, timeperiod=adx_period)

    return adx_values

# QTY

In [3]:
# 사용 가능한 usdt 값
for_usdt = session.get_wallet_balance(
    accountType="CONTRACT",
    coin="USDT",
)

available_usdt = for_usdt['result']['list'][0]['coin'][0]['availableToWithdraw']
available_usdt = float(available_usdt)

# 최소 주문 금액에 따른 소수점 버리기 및 정수화
session = HTTP(testnet=True)
instruments_info = session.get_instruments_info(
    category="linear",
    symbol="BTCUSDT",
)

minOrderQty = instruments_info['result']['list'][0]['lotSizeFilter']['minOrderQty']
minOrderQty = float(minOrderQty)

if minOrderQty == 0.001:
    result_qty = available_usdt / 현재 종가
    result_qty = "{:.3f}".format(result_qty)

elif minOrderQty == 0.01:
    result_qty = available_usdt / 현재 종가
    result_qty = "{:.2f}".format(result_qty)

elif minOrderQty == 0.1:
    result_qty = available_usdt / 현재 종가
    result_qty = "{:.1f}".format(result_qty)

else:
    result_qty = available_usdt / 현재 종가
    result_qty = int(result_qty)
    result_qty = str(result_qty)




주문 시간 :  2023-11-22 13:44:18.766000


# 주문

In [1]:
order_execution = session.place_order(
    category="linear",
    symbol="MNTUSDT",
    side="Buy", 
    orderType="Market",
    qty=result_qty, 
    timeInForce="IOC", 
    isLeverage=1,
    orderFilter="Order",
    
    # TP / SL
    takeProfit = tp_price
    stopLoss = sl_price
)

oder_time = datetime.utcfromtimestamp(order_execution['time']/1000)

print(f'주문 시간 : {oder_time} /  주문량 : {result_qty} / TP : {tp_price} / SL : {sl_price}')

'1.242'

# TP / SL

In [29]:
def find_first_digit_after_decimal(str_number):
    # 소수점 이하의 숫자 추출
    decimal_part = str(str_number).split('.')[1]
    
    # 1의 자리 숫자 찾기
    for i in range(len(decimal_part)):
        if decimal_part[i] == '1':
            return i + 1  # 1의 자리이므로 인덱스 + 1 반환
    
    return None  # 1의 자리 숫자가 없는 경우

# 최소 주문 금액
minPrice = instruments_info['result']['list'][0]['priceFilter']['minPrice']

# 1의 자리수 
one_seat = find_first_digit_after_decimal(minPrice)

# TP / SL 변수 값
tp_price = 현재 종가 + (현재 종가 * 0.01)
tp_price = str(tp_price)
tp_price = tp_price[:one_seat+2]
tp_price = float(tp_price)

sl_price = 현재 종가 + (현재 종가 * 0.005)
sl_price = str(sl_price)
sl_price = sl_price[:one_seat+2]
sl_price = float(sl_price)

# TEST